**Reference**:

https://towardsdatascience.com/how-to-fine-tune-an-nlp-regression-model-with-transformers-and-huggingface-94b2ed6f798f

https://predictivehacks.com/how-to-fine-tune-an-nlp-regression-model-with-transformers-and-huggingface/


# Don't normalize the data 

# Linear Regression 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from transformers.utils import logging
logging.set_verbosity_error()
pd.set_option('display.max_colwidth', None)

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer


In [2]:
! pwd

/Users/yousef/code/thesis-nlp/code-wip-dataset


In [3]:
FILE_PATH = '/Users/yousef/code/thesis-nlp/dataset/incident-ohlc-v06-scalar.csv'
COLUMNS =  ['w_open', 'w_high', 'w_low', 'w_close', 'new', 'started','done', 'dayofweek', 'dayofmonth', 'dayofyear', 'target', 'y_ystrdy']

# N_ROWS = 100
# df_ = pd.read_csv(FILE_PATH,parse_dates=['date'],index_col=['date'], nrows=N_ROWS)
df_ = pd.read_csv(FILE_PATH,parse_dates=['date'],index_col=['date'])
df_ = (df_.sort_index())[COLUMNS]
df_ = df_.iloc[1:-1]  # Remove NaN

max_number_in_data = int(df_.describe().transpose()["max"].max())
print(f'Max number in the data is {max_number_in_data}')
display(df_.head(2))

df = pd.DataFrame()
df['label']= df_['target']
df['text']= ''
for col in COLUMNS[:-2]:
    df_[col] = df_[col].apply(lambda x: f'{x:.0f}')
    df['text'] = df['text'] + f' {col} ' + df_[col].astype(str)

display(df.head(2))

Max number in the data is 366


,w_open,w_high,w_low,w_close,new,started,done,dayofweek,dayofmonth,dayofyear,target,y_ystrdy
date,,,,,,,,,,,,
2010-01-14,0.0,1.0,0.0,1.0,3.0,1.0,0.0,3,14,14,1.0,0.0
2010-01-15,1.0,1.0,1.0,1.0,1.0,0.0,0.0,4,15,15,1.0,2.0


,label,text
date,,
2010-01-14,1.0,w_open 0 w_high 1 w_low 0 w_close 1 new 3 started 1 done 0 dayofweek 3 dayofmonth 14 dayofyear 14
2010-01-15,1.0,w_open 1 w_high 1 w_low 1 w_close 1 new 1 started 0 done 0 dayofweek 4 dayofmonth 15 dayofyear 15


In [4]:
df.iloc[6]

label                                                                                                   6.0
text      w_open 3 w_high 4 w_low 3 w_close 4 new 0 started 1 done 0 dayofweek 2 dayofmonth 20 dayofyear 20
Name: 2010-01-20 00:00:00, dtype: object

In [5]:
# ','.join(df[['text']])
# df['text']
# ' '.join(df['text'][0:3].to_list())
# # df['label'][0]

## Create and customize AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
for x in COLUMNS:
    tokenizer.add_tokens(x)
    
for x in range(max_number_in_data):
    tokenizer.add_tokens(str(x))
    
        
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


## Pandas To Dataset

In [7]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from datasets import Dataset


df_lb = pd.DataFrame()
df_lb['label'] = ''
df_lb['text'] = ''

LOOKBACK_WINDOW =5
for i in range(LOOKBACK_WINDOW, len(df)):
    text = ' '.join(df['text'][i - LOOKBACK_WINDOW:i].to_list())
    label = df['label'][i]
    df_lb.loc[len(df_lb.index)] = [label,text] 

datasets = []
N_SPLIT = 10
tscv = TimeSeriesSplit(n_splits=N_SPLIT)
for train_index, test_index in tscv.split(df_lb):
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    df_tr = df_lb.iloc[train_index]
    df_ts = df_lb.iloc[test_index]
    tr_ds = Dataset.from_pandas(df_tr, split="train",preserve_index=False)
    ts_ds = Dataset.from_pandas(df_ts, split="test",preserve_index=False)
    datasets.append({'train':tr_ds,'test':ts_ds})

print(f'length of datasets is {len(datasets)}')
print({datasets[0]['train']})
print({datasets[0]['test']})

TRAIN: 135 TEST: 131
TRAIN: 266 TEST: 131
TRAIN: 397 TEST: 131
TRAIN: 528 TEST: 131
TRAIN: 659 TEST: 131
TRAIN: 790 TEST: 131
TRAIN: 921 TEST: 131
TRAIN: 1052 TEST: 131
TRAIN: 1183 TEST: 131
TRAIN: 1314 TEST: 131
length of datasets is 10
{Dataset({
    features: ['label', 'text'],
    num_rows: 135
})}
{Dataset({
    features: ['label', 'text'],
    num_rows: 131
})}


## Fine-Tuning The Model

### Metrics Function


In [8]:
from datasets import load_metric
from sklearn.metrics import mean_squared_error
import evaluate

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # rmse = mean_squared_error(labels, predictions, squared=False)
    rmse = mean_squared_error(labels, predictions, squared=True)
    return {"rmse": rmse}

def compute_metrics_mape(eval_pred):
    predictions, labels = eval_pred
    mape = mape_metric.compute(predictions=predictions, references=labels)
    return {"mape": mape}

In [9]:
# num_labels =1 means regression
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)
model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False,
                                  report_to="none",
                                  optim="adamw_torch",
                                  # optim="adagrad"
                                  

                                  )

### Train The Model

In [ ]:
train_output = []
test_output = []

transformers.utils.logging.set_verbosity_error()

for idx, ds in enumerate(datasets):
    tokenized_train_ds = ds['train'].map(tokenize_function, batched=True)
    tokenized_test_ds = ds['test'].map(tokenize_function, batched=True)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_ds,
        eval_dataset=tokenized_test_ds,
        compute_metrics=compute_metrics
    )
    print(f'\n<= Training {idx} of {len(datasets)} .... =>')
    trainer.train()
    train_output.append(trainer.evaluate(tokenized_train_ds))
    test_output.append(trainer.evaluate())
print('\n<= Train is done =>')


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 135
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27
  Number of trainable parameters = 66969601



<= Training 0 of 10 .... =>


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 131
  Batch size = 16


...

{'eval_loss': 12176.06640625, 'eval_rmse': 12176.06640625, 'eval_runtime': 68.0673, 'eval_samples_per_second': 7.757, 'eval_steps_per_second': 0.485, 'epoch': 3.0}
{'eval_loss': 17657.205078125, 'eval_rmse': 17657.203125, 'eval_runtime': 16.8956, 'eval_samples_per_second': 7.754, 'eval_steps_per_second': 0.533, 'epoch': 3.0}


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 659
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 126
  Number of trainable parameters = 66969601



<= Training 4 of 10 .... =>


In [ ]:
# tr_loss = [x.training_loss for x in train_output]
tr_loss = [x['eval_loss'] for x in train_output]
ts_loss = [x['eval_loss'] for x in test_output]
plt.plot(tr_loss,label='tr_loss')
plt.plot(ts_loss,label='ts_loss')
plt.legend()

In [ ]:
# tr_loss = [x.metrics['train_runtime'] for x in train_output]
ts_loss = [x['eval_rmse'] for x in train_output]
ts_loss = [x['eval_rmse'] for x in test_output]
plt.plot(tr_loss,label='tr_rmse')
plt.plot(ts_loss,label='ts_rmse')
plt.legend()

In [ ]:
train_output[0]

In [ ]:
# tokenized_test_ds['text']
# tokenized_test_ds['text']


# Train and test chart and metrics

In [ ]:
print('\n<= Predicting =>')
predictions_test = trainer.predict(tokenized_datasets["test"])
predictions_train = trainer.predict(tokenized_datasets["train"])
print('\n<= Predictions are done =>')

In [ ]:
data = [predictions_train.predictions,predictions_test.predictions]
fig, axs = plt.subplots(nrows=1,ncols=2,figsize=(10, 5))

axs[0].plot(data[0],label='train')
axs[0].plot(dataset["train"]['label'],label='y')
axs[0].title.set_text('train ')
axs[0].legend(loc='upper right')


axs[1].plot(data[1],label='test')
axs[1].plot(dataset["test"]['label'],label='y')
axs[1].title.set_text('test ')
axs[1].legend(loc='upper right')


In [ ]:
plt.plot(predictions_test.predictions)

In [ ]:
print(f'key\t\t train.metric \t test.metrics')

for key in list(predictions_test.metrics.keys())[:2]:
    print(f'{key} \t\t{predictions_train.metrics[key]:0.3f} \t\t{predictions_test.metrics[key]:0.3f}')

## Save And Load The Pre-Trained Model And Tokenizer


In [ ]:
# save the model/tokenizer

model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

# load the model/tokenizer

from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

##  Use The Model

In [ ]:
from transformers import Trainer
trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

pipeline_prediction("🚨 Get 50% now!")